In [15]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold

In [16]:
files_path = 'CleanFiles/'
files_format = '.csv'
files_postfix = 'Extract'
file_names = ['AgiaParaskevi','Aristotelous','Athens','Elefsina','Koropi','Liosia',
              'Lykovrisi','Marousi','NeaSmirni','Patision','Peristeri',
              'Pireus','Thrakomakedones']

In [17]:
def pm10_index(val):
    if val <= 25.0:
        return 0
    elif 26.0 <= val <= 50.0:
        return 1
    elif 51.0 <= val <= 90.0:
        return 2
    elif 91.0 <= val <= 180.0:
        return 3
    else:
        return 4

In [18]:
def get_time(tmp):
    return tmp[11:13]

In [71]:
#neasmirni = pd.read_csv('CleanFiles/NeaSmirniExtract.csv', sep=',')
#neasmirni = neasmirni[['date_time','station_id','season','real_temp','real_windspd','real_winddir','real_humidity','forecast_tempC','forecast_windSpeed','forecast_windDirection','forecast_humidity','pm10']]
#neasmirni['pm10'] = neasmirni['pm10'].map(lambda a: pm10_index(a))
#temp = pd.read_csv('CleanFiles/PeristeriExtract.csv', sep=',')

In [20]:
# T = 72
# D = 1
# X = []
# Y = []
# for t in range(len(neasmirni) - T):
#     x = neasmirni[t:t+T]
#     X.append(x)
#     y = neasmirni[t:t+T]
#     Y.append(y)

In [83]:
def create_xy(series, series2, window_size, prediction_horizon, shuffle = False):
    x = []
    y = []
    for i in range(0, len(series)):
        if len(series[(i + window_size):(i + window_size + prediction_horizon)]) < prediction_horizon:
            break
        x.append(np.array(series[i:(i + window_size)]))
        y.append(np.array(series2[(i + window_size):(i + window_size + prediction_horizon)]))
    x = np.array(x)
    y = np.array(y)
    return x,y

In [84]:
parameters = {
    'n_estimators': 2000,
    'max_depth': 4,
    'num_leaves': 2**4,
    'learning_rate': 0.1,
    'boosting_type': 'dart'
}

In [87]:
def loadToModel(files, formating, pathFiles, postfx, params):
    T = 72
    D = 1
    X = []
    Y = []
    for fileName in files:
        accuracy = 0
        df = pd.read_csv(pathFiles + fileName + postfx + formating, sep=',')
        df['pm10'] = df['pm10'].map(lambda a: pm10_index(a))
        df['date_time'] = df['date_time'].astype("|S")
        df['season'] = df['season'].astype("|S")
        df['forecast_windDirection'] = df['forecast_windDirection'].astype("|S")
        df['real_winddir'] = df['real_winddir'].astype("|S")
        #df = df[['date_time','station_id','season','real_temp','real_humidity','real_windspd','real_winddir','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection']]
#         for t in range(len(df) - T):
#             X.append(df[t:t+T][['date_time','station_id','season','real_temp','real_humidity','real_windspd','real_winddir','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection']])
#             Y.append(df[t:t+T][['pm10']])
#         cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
#         n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
        X,y = create_xy(df[['date_time','station_id','season','real_temp','real_humidity','real_windspd','real_winddir','forecast_tempC','forecast_humidity','forecast_windSpeed','forecast_windDirection']], df[['pm10']],72, 1)
        accuracy = 0
        print(df.dtypes)
        print(fileName + ' begins prediction')
        model = LGBMClassifier()
        kf = KFold(n_splits=5)
        for trainI,testI in kf.split(X):
            model.fit(X[trainI],y[trainI])
            predictions = model.predict(X[testI])
            accuracy += accuracy_score(y[testI], predictions)
        
        print('Accuracy ' + fileName + ': %.3f' % (accuracy))

In [88]:
loadToModel(file_names, files_format, files_path, files_postfix, parameters)

Unnamed: 0                  int64
date_time                    |S19
real_temp                 float64
real_humidity             float64
real_windspd              float64
station_id                  int64
forecast_tempC              int64
forecast_humidity           int64
season                       |S13
forecast_windDirection        |S3
forecast_windSpeed        float64
pm10                        int64
pm25                      float64
o3                        float64
no                        float64
no2                       float64
real_winddir                  |S3
dtype: object
AgiaParaskevi begins prediction


ValueError: Unknown label type: 'unknown'